In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from tqdm import tqdm

# Load data
ordini_per_cliente = json.load(open('data/utils/ordini_per_client.json'))       
tot_per_cliente = json.load(open('data/utils/tot_per_client.json')) 
quantity = pd.read_csv('data/utils/quantity_per_client.csv')
print(f'prima {quantity.shape}')

job_machines = json.load(open('data/utils/articoli_macchine.json'))
GROUND_TRUTH = [job for job in job_machines] 

#Filtraggio degli ordini che NON presentano almeno una associazione macchina
o_p_c_filtrata = {}
for cliente in ordini_per_cliente:
    dict_ordini = ordini_per_cliente[cliente] 
    for ordine in dict_ordini:
        if GROUND_TRUTH.count(ordine) > 0:
            if cliente not in o_p_c_filtrata:
                o_p_c_filtrata[cliente] = {}
            # Append the new value to the list associated with the key
            o_p_c_filtrata[cliente][ordine] = ordini_per_cliente[cliente][ordine]
        # else:
        #     print(f'per cliente: {cliente}, ordine: {ordine}, non trovato')

t_p_c_filtrata = {}
for cliente in tot_per_cliente:
    if o_p_c_filtrata.get(cliente) is not None:
        t_p_c_filtrata[cliente] = tot_per_cliente[cliente]

quantity_filtrata = {'Cliente': [], 'cod articolo': [], 'mean': [], 'std': [], 'mean_differenza_giorni': []}
for index, row in quantity.iterrows():
    # print(f"Row {index}: {row.to_dict()}")
    if t_p_c_filtrata.get(row['Cliente']) is not None and GROUND_TRUTH.count(row['cod articolo']) > 0:
        quantity_filtrata['Cliente'].append(row['Cliente'])
        quantity_filtrata['cod articolo'].append(row['cod articolo'])
        quantity_filtrata['mean'].append(row['mean'])
        quantity_filtrata['std'].append(row['std'])
        quantity_filtrata['mean_differenza_giorni'].append(row['mean_differenza_giorni'])

quantity = pd.DataFrame(quantity_filtrata)
print(f'dopo {quantity.shape}')

print(f'prima: {len(ordini_per_cliente)}, dopo {len(o_p_c_filtrata)}')
print(ordini_per_cliente['GT 2000'])
print(o_p_c_filtrata.get('GT 2000'))

print(f'prima: {len(tot_per_cliente)}, dopo {len(t_p_c_filtrata)}')

# Utils
clienti = list(o_p_c_filtrata.keys())               # Lista di clienti
tot = sum(t_p_c_filtrata.values())                     # Totale ordini per cliente nello storico
norm = {k: v/tot for k,v in t_p_c_filtrata.items()}    # Probabilità per cliente di comparire nello storico

prima (316, 5)
dopo (114, 5)
prima: 87, dopo 49
{'2218ZN': 1}
None
prima: 87, dopo 49


In [19]:
norm

{'SANYLEG': 0.17157490396927016,
 'CALZIFICIO TREVER': 0.09475032010243278,
 'DIAMANT': 0.09090909090909091,
 'GIZETA': 0.08578745198463508,
 'CALZIFICIO NUOVO GIODI': 0.0499359795134443,
 'FALKE': 0.04353393085787452,
 'CSP PARIS': 0.04225352112676056,
 'MANIFATTURA ITALIA CUCIRINI': 0.034571062740076826,
 'BBF GAMBETTI': 0.030729833546734954,
 'CALZIFICIO TEX STYLE': 0.028169014084507043,
 'FGP': 0.028169014084507043,
 'MARYGOLD': 0.026888604353393086,
 'CSP INTERNATIONAL': 0.024327784891165175,
 'CALZIFICIO SANBA': 0.020486555697823303,
 'CALZIFICIO CREMONA': 0.020486555697823303,
 'RADICI': 0.019206145966709345,
 'FISHER & PAYKEL': 0.015364916773367477,
 'CALZIFICIO GENNY': 0.014084507042253521,
 'ATHENA': 0.014084507042253521,
 'FRITSCHE': 0.012804097311139564,
 'CALZE LUISA': 0.011523687580025609,
 'CALZIFICIO ARGOPI': 0.011523687580025609,
 'W.ZIMMERMANN': 0.008962868117797696,
 'SCORTA': 0.008962868117797696,
 'UTEXBEL': 0.008962868117797696,
 'FEIN ELAST GRABHER': 0.0076824583

In [36]:
### Generate synthetic data

## params
num_samples = 10        # Number of samples to generate

# Set to True to generate data starting from today (orders in entrata oggi)
today = True       

# Altrimenti imposta un range
start = '2024-10-01'
end = '2024-12-31'

new_data = []

# Loop to generate synthetic data with progress bar
for _ in tqdm(range(num_samples), desc="Generating data"):

    # Scegli cliente in modo uniforme
    i = np.round(np.random.uniform(0, len(clienti) - 1))
    cliente = clienti[int(i)]
    prob = norm[cliente]
    num = int(np.round(prob * num_samples))                     # Numero di ordini da generare per il cliente
    rows = quantity[quantity['Cliente'] == cliente].shape[0]
    if rows > 0:                                                # Se il cliente ha articoli nel dataset
        articoli = quantity[quantity['Cliente'] == cliente]     # Filtra articoli per cliente
        
        for _ in range(num + 1):
            art_index = int(np.round(np.random.uniform(0, rows - 1)))   # Scegli un articolo in modo uniforme
            art = articoli.iloc[art_index]

            articolo = art['cod articolo']              # Nome dell'articolo
            quantity_mean = art['mean']                 # Media della quantità ordinata                
            quantity_std = art['std']                   # Deviazione standard della quantità ordinata
            giorni = art['mean_differenza_giorni']      # media(consegna - ordine) | articolo, cliente

            # Ensure kg is always positive
            kg = np.round(np.random.normal(quantity_mean, quantity_std))
            if kg <= 0:                                     # Se la quantità è negativa, 
                kg = quantity_mean                              # prendi la media
            
            if today:
                data_inizio = pd.Timestamp.today().date()
            else:
                date_range = pd.date_range(start=start, end=end)
                data_inizio = np.random.choice(date_range)
                data_inizio = pd.Timestamp(data_inizio).date()

            # Calculate the end date
            data_fine = data_inizio + pd.Timedelta(days=giorni)
            data_fine = pd.Timestamp(data_fine).date()

            # Append the generated data to new_data list
            new_data.append({                               # Inserisci come dizionario in una lista di ordini
                'cliente': cliente,
                'cod_articolo': articolo,
                'quantity': kg,
                'data inserimento': data_inizio,
                'data consegna': data_fine
            })

columns = ['cliente', 'cod_articolo', 'quantity', 'data inserimento', 'data consegna']
res = pd.DataFrame(data=new_data, columns=columns)

res.to_csv('data/new_orders.csv', index=False)
res

Generating data: 100%|██████████| 10/10 [00:00<00:00, 542.83it/s]

,cliente,cod_articolo,quantity,data inserimento,data consegna
0,CALZIFICIO ATHENA,4463S+ZN1,420.0,2024-11-12,2024-12-12
1,FEIN ELAST GRABHER,10353ZF,380.0,2024-11-12,2024-12-08
2,FISHER & PAYKEL,X21-39599V7DN0332,195.0,2024-11-12,2025-01-09
3,TATRASVIT,S18-200014DN,7.0,2024-11-12,2024-12-08
4,TATRASVIT,S18-200014DN,7.0,2024-11-12,2024-12-08
5,CALZIFICIO GENNY,X23-7892DN mini,196.0,2024-11-12,2024-12-08
6,INVENTEX,S15-160018DN,587.0,2024-11-12,2024-12-02
7,CALZIFICIO BRAM,X24-2001S+ZN,115.0,2024-11-12,2024-11-23
8,FEIN ELAST GRABHER,10353ZF,380.0,2024-11-12,2024-12-08
9,INVENTEX,S15-160018DN,587.0,2024-11-12,2024-12-02
